# 1 
#### Отсюда взял https://github.com/rahulkidambi/AccSGD
    r"""Implements the algorithm proposed in https://arxiv.org/pdf/1704.08227.pdf, which is a provably accelerated method 
    for stochastic optimization. This has been employed in https://openreview.net/forum?id=rJTutzbA- for training several 
    deep learning models of practical interest. This code has been implemented by building on the construction of the SGD 
    optimization module found in pytorch codebase.
    Args:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float): learning rate (required)
        kappa (float, optional): ratio of long to short step (default: 1000)
        xi (float, optional): statistical advantage parameter (default: 10)
        smallConst (float, optional): any value <=1 (default: 0.7)
    Example:
        >>> from AccSGD import *
        >>> optimizer = AccSGD(model.parameters(), lr=0.1, kappa = 1000.0, xi = 10.0)
        >>> optimizer.zero_grad()
        >>> loss_fn(model(input), target).backward()
        >>> optimizer.step()
    """

In [1]:
from torch.optim.optimizer import Optimizer, required
import copy

class AccSGD(Optimizer):

    def __init__(self, params, lr=required, kappa = 1000.0, xi = 10.0, smallConst = 0.7, weight_decay=0):
        defaults = dict(lr=lr, kappa=kappa, xi=xi, smallConst=smallConst,
                        weight_decay=weight_decay)
        super(AccSGD, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(AccSGD, self).__setstate__(state)

    def step(self, closure=None):
        """ Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            weight_decay = group['weight_decay']
            large_lr = (group['lr']*group['kappa'])/(group['smallConst'])
            Alpha = 1.0 - ((group['smallConst']*group['smallConst']*group['xi'])/group['kappa'])
            Beta = 1.0 - Alpha
            zeta = group['smallConst']/(group['smallConst']+Beta)
            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad.data
                if weight_decay != 0:
                    d_p.add_(weight_decay, p.data)
                param_state = self.state[p]
                if 'momentum_buffer' not in param_state:
                    param_state['momentum_buffer'] = copy.deepcopy(p.data)
                buf = param_state['momentum_buffer']
                buf.mul_((1.0/Beta)-1.0)
                buf.add_(-large_lr,d_p)
                buf.add_(p.data)
                buf.mul_(Beta)

                p.data.add_(-group['lr'],d_p)
                p.data.mul_(zeta)
                p.data.add_(1.0-zeta,buf)

        return loss

# 2

* Ощень полезна http://ruder.io/optimizing-gradient-descent/index.html#adagrad

* И тут тоже намана так рассказано https://moodle2.cs.huji.ac.il/nu15/pluginfile.php/316969/mod_resource/content/1/adam_pres.pdf

* Тащемта каефная реализация на торче от китайца того, что нам надо (почти, они там все мироточат на эту https://openreview.net/pdf?id=ryQu7f-RZ статью и поэтому только для нее делают реализации) https://github.com/wikaiqi/AMSGradpytorch/blob/master/AdamBeyond.ipynb

* Вот тута челик почти все, что надо нам для адама и не адама реализовал, но только не на торче, а на Лазанье (итальянец наверное(нет(кто так называет фреймворки???))) https://fdlm.github.io/post/amsgrad/

* Реализация адама без евы на theano (не знаю зачем, в торче делается в одну строчку) https://github.com/aimpast/amsgrad/blob/master/adam.py

* Еще полезные очень похожие на наши эксперименты (опять на лазанье скока итальянцев развелось) https://github.com/fdlm/amsgrad_experiments



# 3

В нашей статье сравниваются результаты работы оптимизаторов Adam и AMSGrad с реализациями новых оптимизаторов из статьи: A2Grad-uni, A2Grad-inc, A2Grad-exp

На чем тестят:
* Logistic regression on MNIST
* Two-layer neural network on MNIST (ReLu в конце, кросс-энтропия)
* Deep neural networks on CIFAR10 (Cifarnet (сверточная сетка) and Vgg16(вроде можно скачать готовые веса))

Парметры:
* Adam и AMSGrad: $\beta_1$ = 0.9, $\beta_2$ из сетки {0.99, 0.999}; Learning rate из сетки {0.0001, 0.001, 0.01, 0.1}, momentum parameter из сетки {0.5, 0.9}

Их результаты: короче, каефно реализовали, смарите пацаны A2Grad круче Адама и ~~Евы~~ AMSGrad